In [1]:
from glob import glob

files = sorted(glob('chatgpt-iium-confession-noisy-translation/*'))
files

['chatgpt-iium-confession-noisy-translation/clean-iium00.splitted.gpt.requested',
 'chatgpt-iium-confession-noisy-translation/clean-iium01.splitted.gpt.requested',
 'chatgpt-iium-confession-noisy-translation/clean-iium02.splitted.gpt.requested',
 'chatgpt-iium-confession-noisy-translation/clean-iium03.splitted.gpt.requested',
 'chatgpt-iium-confession-noisy-translation/iium-confession-gpt.json',
 'chatgpt-iium-confession-noisy-translation/iium-confession-more.json']

In [2]:
!head -n 1 chatgpt-iium-confession-noisy-translation/clean-iium00.splitted.gpt.requested

{"src": {"text": "Assalamualaikum dan salam sejahtera. Hai geng. J is back. Sihat semua? Family sihat? Aku doakan semua sihat-sihat belaka. Terima kasih support aku. Aku cuma nak cerita kisah aku dan kawan-kawan aku. Kerja di ladang ni dah lama di anak tirikan oleh masyarakat. Sedih juga aku. Kami duduk hutan sawit ni tak kacau orang pun. Tapi pandangan masyarakat tu, hurmmm... Entahla."}, "r": {"english": "Assalamualaikum and greetings. Hey guys, J is back. Are you all well? Is your family well? I pray that everyone is in good health. Thank you for your support. I just want to share my story and my friends' story. We have been working in this plantation for a long time and have been neglected by the community. It's sad. We live in this oil palm forest and don't bother anyone. But the community's perception, well... I don't know.", "malay": "Assalamualaikum dan salam sejahtera. Hai geng. J kembali. Semua sihat? Keluarga sihat? Saya berdoa agar semua sihat-sihat belaka. Terima kasih ata

In [3]:
import json

iium_confession = []
for f in files:
    with open(f) as fopen:
        if 'gpt.requested' in f:
            for l in fopen:
                data = json.loads(l)
                iium_confession.append(data)
        else:
            data = json.load(fopen)
            iium_confession.extend(data)
            
iium_confession = [a for a in iium_confession if a is not None]

In [4]:
from tqdm import tqdm

filtered = []
for d in tqdm(iium_confession):
    if 'src' in d:
        left = d['src']['text']
        try:
            en = d['r']['english']
            ms = d['r']['malay']
        except:
            en = None
            ms = None
    else:
        left = d[0]
        success = False
        try:
            r = json.loads(d[1])
            success = True
        except:
            pass
        
        if not success:
            try:
                r = eval(d[1])
                success = True
            except:
                pass
                    
        if success:
            if isinstance(r, tuple):
                en = [r_['english'] for r_ in r]
                ms = [r_['malay'] for r_ in r]
                en = ' '.join(en)
                ms = ' '.join(ms)
            else:
                en = r['english']
                ms = r['malay']
        else:
            en = None
            ms = None
    
    if en or ms:
        filtered.append({
            'left': left,
            'en': en,
            'ms': ms
        })           

100%|████████████████████████████████████████████████████████████████████████████████████| 475485/475485 [00:00<00:00, 780054.79it/s]


In [5]:
left

"Ada satu hari tu, abah bawa aku ikut dia pergi kerja sebab masa tu taka da sesiapa nak jaga aku kat rumah. Masa tu umur aku 6 tahun. Kat tempat kerja abah tu ada kontena kosong, abah letak aku kat situ. Dari situ aku boleh je tengok abah bekerja. Disebabkan abah kerja kat kilang jahit baju, aku masa tu ingatkan abah tukang jahit. Tapi apa yang aku nampak depan mata aku masa tu ialah Abah mencangkung sambil kutip sampah. Ya, Abah aku kerja tukang kutip sampah dekat kilang jahit. Masa tu aku umur 6 tahun, aku tak fikir apa-apa. Kat sekolah, masa kecik aku pandai sikit. Abah tanya periksa dapat nombor berapa. Aku cakap aku dapat nombor 3. Masa tu aku suka sangat makan roti pong (roti yang ada kismis kat tengah). Boleh pulak abah tanya as hadiah nak dia belikan roti pong atau basikal. Eh mestilah aku nak basikal! Haha tapi malangnya masa tu ak kecik lagi so aku tak tahu macam mana susah payahnya abah aku bekerja nak belikan aku basikal.. Ada satu hari abah belikan aku air kotak milo. Aku 

In [6]:
r

{'english': "Since I moved schools when I was 8 years old, I haven't really liked coming to school. You can imagine what learning was like in second grade. We had to draw pictures of our families in class, make cards for Mother's Day, Father's Day, and so on. From second grade until high school, I only gave cards to my teachers. On Mother's Day, I gave a card to my female teacher, and on Father's Day, I gave a card to my male teacher. It was like that until high school when we had Civics and Citizenship as a subject, and even then, I gave a card to my teacher haha. One night, my siblings and I went to our grandfather's house. Our uncle and cousin were there too. Our uncle was the type to lecture us when he was angry, and then he would just sweep the trash. Whenever I went to my grandfather's house, he was always sweeping. Anyway, one night, he got angry at his child and said, 'Don't be like a child without parents, neglected with no future.' I felt it even though I was still young. But

In [7]:
len(filtered), len(iium_confession)

(333758, 475485)

In [8]:
def reject(k):
    if 'saya tidak dapat menterjemah teks' in k.lower():
        return
    if 'saya tidak boleh menterjemah kandungan yang tidak sesuai' in k.lower():
        return
    if 'saya tidak boleh menterjemah teks' in k.lower():
        return
    if 'teks yang diberikan tidak mempunyai makna' in k.lower():
        return
    if 'teks yang disediakan tidak boleh diterjemahkan' in k.lower():
        return
    if 'teks yang diberikan tidak masuk akal' in k.lower():
        return
    if 'teks yang diberikan nampaknya tidak' in k.lower():
        return
    if 'teks yang diberikan tidak mempunyai maksud' in k.lower():
        return
    if 'teks yang diberikan bukan dalam mana-mana bahasa' in k.lower():
        return
    if 'teks ini tidak boleh diterjemahkan' in k.lower():
        return
    if 'saya tidak dapat menterjemah frasa' in k.lower():
        return
    if 'teks yang disediakan bukan' in k.lower():
        return
    if 'teks yang diberikan tidak jelas' in k.lower():
        return
    if 'teks yang diberikan tidak' in k.lower():
        return
    if 'terjemahan teks kepada' in k.lower():
        return
    if 'saya tidak boleh menterjemah bahasa' in k.lower():
        return
    if 'model bahasa AI' in k:
        return
    if 'bahasa melayu standard' in k.lower():
        return
    if 'return JSON structure' in k:
        return
    if 'teks yang diberikan bukan dalam bahasa' in k.lower():
        return
    if 'teks yang anda berikan bukan' in k.lower():
        return
    if 'teks yang disediakan tidak' in k.lower():
        return
    if 'teks yang diberikan bukan' in k.lower():
        return
    if 'saya tidak dapat menterjemah' in k.lower():
        return
    if 'tetapi teks yang diberikan' in k.lower():
        return
    
    return True

In [9]:
!wc -l processed-iium-confession.jsonl

333758 processed-iium-confession.jsonl


In [11]:
with open('processed-iium-confession.jsonl', 'w') as fopen:
    for d in filtered:
        try:
#             if not reject(d['ms']):
#                 print(d)
#                 continue
                # break
            fopen.write(f'{json.dumps(d)}\n')
        except:
            pass